In [9]:
import ipywidgets as widgets
import traitlets

# MVC, object-oriented approach

This last round with our simple (but bad) password generator mostly follows the MVC design introduced in the last notebook, but use classes for the model and view.

We actually won't use strict MVC here; the controller code is quite minimal and in the end we'll include it in the view.

## View: The widget, as a class

This time we will build the password widget as a subclass of `VBox`. This has the virtue of making it easy to distribute and use just like one of `ipywidget`'s built in widgets.

We'll go through a few iterations of this, with the final one called `PassGenGUI`; until we reach that point we'll add version numbers to the names.

In this first version we construct the same interface as in the previous two notebooks.

The easiest to overlook line in the code below is the one calling the superclass's `__init__`. If you forget that then your widget will not work even though you have subclassed from `VBox`. 

Notice that the individual widgets are "private" in the sense that their names are prepended with an underscore. If there are elements that you want to expose to the end user you can, of course, do that.

In [10]:
class PassGenGUI_v1(widgets.VBox):
    def __init__(self):
        # Do NOT forgot to do this ↓↓↓↓↓↓↓↓ 
        super(PassGenGUI_v1, self).__init__()
        
        # Define each of the children...
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('No password yet', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children

Let's take a look at the widget...

In [11]:
pwd_gen = PassGenGUI_v1()
pwd_gen

A Jupyter Widget

Notice that our widget doesn't have a value; it really seems like it should, and that the value should be the password itself. Though we could add that as a `traitlet` linked to the value of the `self._password_text`, we'll add a read-only property to the class instead. The reason is that we don't want users to be able to set the value of our widget's password. Its entire purpose is to generate a passsword for the user.

In [12]:
class PassGenGUI_v2(widgets.VBox):
    def __init__(self):
        # Do NOT forgot to do this!
        super(PassGenGUI_v2, self).__init__()
        
        # Define each of the children...
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('No password yet', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children
        
    @property
    def value(self):
        return self._password_text.value

Let's try it out...

In [13]:
pwd_gen2 = PassGenGUI_v2()
pwd_gen2

A Jupyter Widget

Make sure you can get the value...

In [14]:
pwd_gen2.value

'No password yet'

...and that you cannot *set* the value

In [15]:
pwd_gen2.value = 'new password'

AttributeError: can't set attribute

## An alternative to callbacks: a model with traitlets, eventually linked to the GUI

In the previous iteration of our password generator we wrote a function to generate a password, and a second function to add a callback, i.e. a call to that function whenever a control changes. There is nothing wrong with that approach, but it gets more complicated the more controls you add.

Imagine we add a second control that allows the user to choose whether to include numbers in the password. In addition to modifying the password generating function, we would either need to write a new function for handling changes and make the new numerical checkbox `observe` the new function, or modify `update_password` to check which widget generated the event and make the apropriate call to `calculate_password`.

Instead, we define a class below that generates the password, and includes as class attributes `traitlets` that we will later link to the widget GUI. Note that nothing in this class refers to the widget; it's logic is entirely internal.

To include traitlets a class must subclass from `traitlets.HasTraits`.

As with the GUI, we're going to go through a few iterations, so we'll version them for now.

In [16]:
# Subclass from HasTraits
class PassGen_v1(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    
    def __init__(self):
        # initialize the class by calling the superclass
        super(PassGen_v1, self).__init__()
        
    # The observe decorator is used to indicate that the function being
    # decorated should be called if any of the traits listed as arguments
    # change. The decorated function MUST have an argument named change;
    # in this example the change variable isn't used because we want to update
    # the generated password no matter what the change was.
    
    # Note that we are monitoring changes in PassGen's **own trait**, length,
    # rather than a change in one of the controls in the GUI.
    
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import secrets
    
        new_password = ''.join(secrets.choice(string.ascii_letters) for _ in range(self.length))
        
        # We don't return anything (explanation later). For now just print the 
        # password to observe what happens when we change the length.
        print('In password generator password is: ', new_password)

We'll make an instance of the class so we can see what happens when we change the `length` attribute. 

In [17]:
p = PassGen_v1()

Try changing the length property of in the cell below. Note that any time it is changed, a new password is generated.

In [18]:
p.length = 20

In password generator password is:  eCyrVEWLYZNasjeCMQqW


We don't actually want to print the password out each time we generate a new one, of course, and we eventually want to connect this to a widget-based GUI. The class below adds a new trait for the generated password, and sets that trait in `calculate_password`. You could, in addition, return the generated password; that could be useful in testing your model code, for example.

In [19]:
# Subclass from HasTraits
class PassGen_v2(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    
    # The new trait is defined here:
    password = traitlets.Unicode()
    
    def __init__(self):
        super(PassGen_v2, self).__init__()
            
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import secrets
    
        new_password = ''.join(secrets.choice(string.ascii_letters) for _ in range(self.length))
        
        # Set the value of the password trait here:
        self.password = new_password

Now let's try it out:

In [20]:
p2 = PassGen_v2()

In [21]:
p2.length = 110
p2.password

'eGjIBAILwmfwIPywDBDtfMnahrcXfGsvwcPtYqoVSjcSLUSZYVgETxxCFXkDpEcgyPIqMkisDdkJSbccvIVTvWKlJeVQvZvjRUMmjMCdpGlJeM'

Notice that as currently written nothing prevents setting a nonsensical length:

In [22]:
p2.length = -14
p2.password

''

Traitlets provides a mechanism for validating values using the `@validate` decorator. The documentation on validation is [here](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#validation), but the essential ideas are illustrated below. 

This is the final version of `PassGen`, so we drop the version at the end. 

Only the new lines are commented.

In [6]:
class PassGen(traitlets.HasTraits):
    length = traitlets.Integer()
    password = traitlets.Unicode()
    
    def __init__(self):
        super(PassGen, self).__init__()
            
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import secrets
    
        new_password = ''.join(secrets.choice(string.ascii_letters) for _ in range(self.length))
        
        # Set the value of the password trait here:
        self.password = new_password

    # The new validator:
    @traitlets.validate('length')
    # You can name this method anything you want.
    def _validate_length(self, proposal):
        # proposal contains the new value
        length = proposal['value']
        
        # Test the value
        if length < 1:
            # if it is a bad value, raise a TraitError
            raise traitlets.TraitError('Password length should be positive.')
        # If the value is good, return it.
        return proposal['value']

In [7]:
p3 = PassGen()

Try setting a negative length in the cell below:

In [8]:
p3.length = -23

TraitError: Password length should be positive.

## Controller (or at least modified view)

Now that we have an interface and a model to encapsulate the calcualtion we want to do, we need to connect them. We will do it by creating the links in the GUI class. The code below is identical to `PassGenGUI_v2` from above, with the new lines explained in comments.

It would be fine to complete separate the control code into a separate class. Whichever class contains the control code is the class the user should import.

This is the final version of the `PassGenUI` so we drop the version from the end.

In [21]:
class PassGenGUI(widgets.VBox):
    def __init__(self):
        super(PassGenGUI, self).__init__()
        
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children
        
        # NEW: Create an instance of our model.
        self.model = PassGen()
        
        # NEW: link the password from the model to the _password_text widget
        traitlets.link((self.model, 'password'), (self._password_text, 'value'))
        
        # NEW: link the _password_length widget to the length in the model
        traitlets.link((self.model, 'length'), (self._password_length, 'value'))
        
    @property
    def value(self):
        return self._password_text.value

Let's create an instance of the class, display it, and try it out!

In [22]:
password_widget = PassGenGUI()
password_widget

A Jupyter Widget